In [1]:
from langchain.llms.openai import OpenAI

# gpt 3.5 사용
from langchain.chat_models import ChatOpenAI, ChatAnthropic

llm = OpenAI()
chat = ChatOpenAI()

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

a, b

('\n\nThere are eight planets in our Solar System: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'As of now, there are eight recognized planets in our solar system. These include Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, the definition of a planet has been debated, and there may be other celestial bodies in our solar system that could potentially be classified as planets in the future.')

In [1]:
from langchain.chat_models import ChatOpenAI


# temperature 모델의 창의성 수치를 정할수 있음
# 값이 높을수록 창의성이 증가함, 원하는 값이 등장 안할수도 있음 무작위성 증가
chat = ChatOpenAI(temperature=0.1)

In [3]:
# HumanMessage 는 사람이 보내는 메세지
# AIMessage 는 ai에 의해 보내지는 것
# SystemMessage는 시스템에 설정값을 보낼때 사용함
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]
# 변수들을 지정해서 원하는 값들 넣고 활용 
# messages = [
#     SystemMessage(content="You are a geography expert. And you only reply in {language}."),
#     AIMessage(content="Ciao, mi chiamo {name}!")
#     HumanMessage(content="What is the distance between {contry} and Thailand. Also, what is your name?")
# ]

chat.predict_messages(messages=messages)


AIMessage(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')

In [4]:
chat.predict("Hello, what is your name?")

"Hello! I am an AI language model developed by OpenAI, and I don't have a personal name. You can simply refer to me as OpenAI Assistant. How can I assist you today?"

In [5]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You're a teacher and present your score"),
    AIMessage(content="I am a GPT teacher!"),
    HumanMessage(content="Sir, what is 30 plus 20?")
]


chat.predict_messages(messages=messages)

AIMessage(content='30 plus 20 equals 50.')

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate



# Prompt 기능 사용, 사용자가 오류됀 값을 입력하는걸 방지하기 위해, 아래처럼 두개의 단어만 넣는다
template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")
prompt = template.format(country_a="Maxico", country_b="Korea")

# chat = ChatOpenAI()
chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)

'The distance between Mexico and South Korea is approximately 8,900 miles (14,320 kilometers) when measured in a straight line.'

In [6]:
messages = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a geography expert. And you only reply in {language}.",
        ),
        (
            "ai",
            "Ciao, mi chiamo {name}!",
        ),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also, what is your name?",
        ),
    ]
)


prompt = messages.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate


chat = ChatOpenAI(temperature=0.1)


In [16]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items =  text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse("Hello, How, are you")

['Hello', 'How', 'are you']

In [20]:
template = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT reply with anything else.",
    ),
    ("human", "{question}",),],
)

prompt = template.format_messages(max_items=10, question="What are the colors?",)
result = chat.predict_messages(prompt)

# 리스트로 바꾸는 parser
p = CommaOutputParser()
p.parse(result.content)

['Red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [21]:
# 이것이 체인
chain_one = template | chat | CommaOutputParser()
# chain_two = template | chat | CommaOutputParser()
# all = chain_one | chain_two | parser 이렇게도 됌

chain.invoke({
    "max_items":5,
    "question":"What are the pokemons?",
},)

['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# 스트리밍 작업
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingreients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

In [25]:
# 모르겠다면 모르겠다고 말하는걸 세팅하는게 아주 중요함
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."
        ),
        (
            "human",
            "{recipe}",
        ),
    ],
)

veg_chain = veg_chef_prompt | chat

# recipe 를 담아
final_chain = {"recipe" : chef_chain} | veg_chain


final_chain.invoke(
    {
    "cuisine":"indian",
    },
    )

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's make a classic dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 tbsp garam masala
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1 tsp turmeric
- 1/2 tsp cayenne pepper (adjust to taste)
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (to serve)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, 1 tbsp vegetable oil, half of the minced garlic, half of the grated ginger, 1 tsp garam masala, 1/2 tsp cumin, 1/2 tsp coriander, 1/2 tsp turmeric, cayenne pepper, salt, and pepper. Add the chicken pieces and marinate for at least 1 hour, preferably overn

AIMessageChunk(content="As a vegetarian chef, I can help you make a vegetarian version of Chicken Tikka Masala by replacing the chicken with a suitable alternative. Here's how you can modify the recipe:\n\nAlternative Ingredient:\n- Instead of chicken, you can use paneer (Indian cottage cheese) or tofu as a vegetarian protein option. Both paneer and tofu have a mild flavor that absorbs the spices well, making them great substitutes for chicken in this dish.\n\nPreparation:\n- If using paneer: Cut the paneer into bite-sized cubes and follow the same marination process as you would with the chicken. Paneer doesn't need as much time to marinate, so 30 minutes should be sufficient.\n- If using tofu: Drain the tofu well and cut it into bite-sized pieces. To give the tofu a firmer texture, you can press it between paper towels with a weight on top for about 15-20 minutes before marinating. Tofu can be more delicate, so handle it gently during the marination process.\n\nFollow the rest of the

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# 스트리밍 작업
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(),],
)

t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"]
)

t.format(country="France")

'What is the capital of France'

In [2]:
# FewShot이란 모델에게 예제들을 주는것
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
                    Here is what I know:
                    Capital: Paris
                    Language: French
                    Food: Wine and Cheese
                    Currency: Euro
                    """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
                    I know this:
                    Capital: Rome
                    Language: Italian
                    Food: Pizza and Pasta
                    Currency: Euro
                    """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
                I know this:
                Capital: Athens
                Language: Greek
                Food: Souvlaki and Feta Cheese
                Currency: Euro
                """,
    },
]

chat.predict("What do you know about Franch?")

Franch is a term that is often used to refer to the French language or culture. It is a colloquial term that is sometimes used in a humorous or exaggerated way to describe things that are associated with France or French culture. It is not a formal or official term, but rather a slang term that is used in casual conversation.

'Franch is a term that is often used to refer to the French language or culture. It is a colloquial term that is sometimes used in a humorous or exaggerated way to describe things that are associated with France or French culture. It is not a formal or official term, but rather a slang term that is used in casual conversation.'

In [4]:
example_template = """
    Human: {question}
    AI: {answer}
"""

# 2개다 상관없음
example_prompt = PromptTemplate.from_template(example_template)
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# prompt.format(country="Germany")
chain = prompt | chat

chain.invoke({
    "country":"Germany"
})

AI: 
                I know this:
                Capital: Berlin
                Language: German
                Food: Bratwurst and Sauerkraut
                Currency: Euro

AIMessageChunk(content='AI: \n                I know this:\n                Capital: Berlin\n                Language: German\n                Food: Bratwurst and Sauerkraut\n                Currency: Euro')